In [1]:
"""In this notebook, we clean the SBAnational dataset, 
## to allow for easier data exploration and analysis"""

'In this notebook, we clean the SBAnational dataset, \n## to allow for easier data exploration and analysis'

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("SBAnational.csv", dtype={'ApprovalFY': str, 'ApprovalDate': str, 'DisbursementDate': str},
                 parse_dates=['ApprovalDate', 'DisbursementDate'])
print(df.shape)
print(df.info())
print(df.head())

(899164, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   LoanNr_ChkDgt      899164 non-null  int64         
 1   Name               899150 non-null  object        
 2   City               899134 non-null  object        
 3   State              899150 non-null  object        
 4   Zip                899164 non-null  int64         
 5   Bank               897605 non-null  object        
 6   BankState          897598 non-null  object        
 7   NAICS              899164 non-null  int64         
 8   ApprovalDate       899164 non-null  datetime64[ns]
 9   ApprovalFY         899164 non-null  object        
 10  Term               899164 non-null  int64         
 11  NoEmp              899164 non-null  int64         
 12  NewExist           899028 non-null  float64       
 13  CreateJob          899164 non-n

In [4]:
# Checking for duplicate rows
df[df.duplicated(['LoanNr_ChkDgt'])]
# There are no duplicate rows

# Checking for empty rows
print(df.isnull().all(axis=1).sum())
# There are no empty rows

# Checking for empty columns
print(df.isnull().all(axis=0).sum())
# There are no empty columns

0
0


In [5]:
# Checking for null values in columns
print(df.isnull().sum())
# We will now investigate/correct these

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64


In [6]:
# Evaluating the rate of nulls in ChgOffDate column
print(df.ChgOffDate.isnull().sum() / (df.ChgOffDate.notnull().sum() + df.ChgOffDate.isnull().sum()))
# As ~80% of the values of the column are null, we discard it
df.drop('ChgOffDate', axis=1, inplace=True)

0.8190552557709161


In [7]:
# Investigating NewExist
print(df.groupby('NewExist').LoanNr_ChkDgt.count().reset_index())

# Everything is fine; simply replace null values in NewExist with 0, where 0 = 'Unknown'
df['NewExist'].fillna(0, inplace=True)

# As there are a small amount of unknown rows, we drop them altogether. Also, for uniformity with other
# variables, we map 2 -> 1 and 1 -> 0
df.drop(df[df.NewExist < 0.5].index, inplace=True)
df.NewExist = df.NewExist.apply(lambda x: x - 1).astype(int)

   NewExist  LoanNr_ChkDgt
0       0.0           1034
1       1.0         644869
2       2.0         253125


In [8]:
# Investigating RevLineCr
print(df.groupby('RevLineCr').LoanNr_ChkDgt.count().reset_index())

# 0, Y, N dominate - replace other values with 0, where 0 = 'Unknown'
df.RevLineCr.fillna(0, inplace=True)
df.loc[~df.RevLineCr.isin(['Y', 'N']), 'RevLineCr'] = '0'

   RevLineCr  LoanNr_ChkDgt
0          ,              1
1          -              1
2          .              1
3          0         257546
4          1             22
5          2              6
6          3              1
7          4              1
8          5              1
9          7              1
10         A              1
11         C              2
12         N         419258
13         Q              1
14         R             14
15         T          15277
16         Y         201324
17         `             11


In [9]:
# Investigating LowDoc
print(df.groupby('LowDoc').LoanNr_ChkDgt.count().reset_index())

# Similarly to above, we change the values that aren't Y or N to 0, where 0 = Unknown
df.LowDoc.fillna(0, inplace=True)
df.loc[~df.LowDoc.isin(['Y', 'N']), 'LowDoc'] = '0'

  LowDoc  LoanNr_ChkDgt
0      0           1490
1      1              1
2      A            497
3      C            754
4      N         781786
5      R             75
6      S            602
7      Y         110210


In [10]:
# Investigating MIS_Status
print(df.groupby('MIS_Status').LoanNr_ChkDgt.count().reset_index())

# There are no surprising values here. Our analysis relies on MIS_Status being non-null,
# so we drop all rows that are null in this column
df = df.dropna(subset='MIS_Status')

  MIS_Status  LoanNr_ChkDgt
0     CHGOFF         157481
1      P I F         738524


In [11]:
# Fill name nulls with Unknown
df.Name.fillna('Unknown', inplace=True)

In [12]:
# Checking the relationship between null values in Bank and BankState
temp1 = df[(df['Bank'].isnull() | df['BankState'].isnull())]
print(len(temp1))

# BankState being null implies Bank is null. Let's examine rows where BankState is null but Bank isn't.
temp2 = df[(df['Bank'].notnull() & df['BankState'].isnull())]
print(temp2['Bank'])

# These are foreign banks; let's replace these nulls with 'Outside_USA', fill the rest with 'Unknown' / 'UN'
df.loc[(df['Bank'].notnull() & df['BankState'].isnull()), 'BankState'] = 'Outside_USA'
df.Bank.fillna('Unknown', inplace=True)
df.BankState.fillna('UN', inplace=True)

# Rechecking Bank
print(df.groupby('BankState').LoanNr_ChkDgt.count().reset_index())
# In fact, for our later analysis it's best to class all non-US states as 'UN'
state_abbrev_ls = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
df.loc[~df.BankState.isin(state_abbrev_ls), 'BankState'] = 'UN'

1511
145106               KOREA EXCHANGE BANK
424311    STANDARD CHARTERED BK KOREA LT
444679      HONGKONG & SHANGHAI BK. CORP
447971    STANDARD CHARTERED BK KOREA LT
674674      HONGKONG & SHANGHAI BK. CORP
793950    THE BANK OF EAST ASIA, LIMITED
859233    THE BANK OF EAST ASIA, LIMITED
Name: Bank, dtype: object
      BankState  LoanNr_ChkDgt
0            AK           1434
1            AL          12118
2            AN              1
3            AR           6297
4            AZ           3952
5            CA         117933
6            CO          10561
7            CT           9880
8            DC           3995
9            DE          24498
10           EN              1
11           FL          13793
12           GA          13789
13           GU              4
14           HI           2981
15           IA           9849
16           ID           4310
17           IL          65805
18           IN           7612
19           KS           8856
20           KY           4365


In [13]:
# Similarly, let's examine where State and City are unknown, and see if we can fill the gaps
print(len(df[(df['City'].isnull() & df['State'].isnull())]))

0


In [14]:
# These sets are disjoint - however, given the city, we could fill in the state:
print(df[(df['City'].notnull() & df.State.isnull())]['City'])

264664               CAMERON PARK
306274           BOX 267, APO  AP
328526                    WICHITA
351072    1542,TABUK,SAUDI ARABIA
366139         PALM BEACH GARDENS
366158                      CASCO
367007                   SOMERSET
379174             SALT LAKE CITY
385418                 LAKE OZARK
869948                 A452638533
871847                 A452638533
885335                 A452638533
Name: City, dtype: object


In [15]:
# Investigating these potential values
print(df[(df["City"].str.startswith('A4', na=False))]['City'])  # These values are erroneous
print(df[(df.City.str.contains('BOX 267', na=False))]['City'])  # This value is erroneous
print(df[(df.City.str.contains('CASCO', na=False))]['City'])  # We cannot deduce the state of 'CASCO'

869948    A452638533
871847    A452638533
885335    A452638533
Name: City, dtype: object
306274    BOX 267, APO  AP
Name: City, dtype: object
22211              CASCO
131394             CASCO
159316       SOUTH CASCO
301335       SOUTH CASCO
324947       SOUTH CASCO
349249             CASCO
366158             CASCO
514509       SOUTH CASCO
529930             CASCO
646547             CASCO
664110             CASCO
701499             CASCO
748728             CASCO
750701             CASCO
775304             CASCO
779109             CASCO
859154             CASCO
872787    CASCO TOWNSHIP
Name: City, dtype: object


In [16]:
# We replace the A4... entries and the 'BOX 267' entry with nulls, and fill in states that we know.
df.loc[df.City.str.startswith('A4', na=False), 'City'] = None
df.loc[df.City.str.contains('BOX 267', na=False), 'City'] = None
key_ls = df[(df['City'].notnull() & df.State.isnull())]['LoanNr_ChkDgt'].tolist()
state_ls = ['NJ', 'NY', 'CA', 'KS', None, 'FL', None, 'NJ', 'UT', 'MO']
for i in range(len(key_ls)):
    df.loc[df.LoanNr_ChkDgt == key_ls[i], 'State'] = state_ls[i]

In [17]:
# Finally, we convert the remaining nulls to 'Unknown' / 'U'
df.City.fillna('Unknown', inplace=True)
df.State.fillna('U', inplace=True)

In [18]:
# Converting currency datatypes to floats to make calculations easier
currency_cols = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
for column in currency_cols:
    df[column] = df[column].replace('[\$,]', '', regex=True).astype(float)

In [19]:
# Converting ApprovalFY years to int; however, these is an issue...
print(df.loc[df.ApprovalFY.str.contains('\D')].ApprovalFY)

# The letter 'A' erroneously appears in some years - let's remove them
df.ApprovalFY = df.ApprovalFY.replace('A', '', regex=True).astype(int)

699732    1976A
704030    1976A
705375    1976A
710381    1976A
713245    1976A
748029    1976A
751519    1976A
769515    1976A
775002    1976A
775430    1976A
775978    1976A
776367    1976A
780120    1976A
781090    1976A
784351    1976A
788539    1976A
788661    1976A
793733    1976A
Name: ApprovalFY, dtype: object


In [20]:
# Data cleaning done; saving as csv to import later
"""df.to_csv('SBAnational_cleaned.csv', index=False)"""

"df.to_csv('SBAnational_cleaned.csv', index=False)"